In [2]:
# # setup a client
# import chromadb
# client = chromadb.Client()

# create a collection (like a table)
collection1 = client.create_collection(name="my_collection1")


In [4]:
client.delete_collection("my_collection1")

In [5]:
import pandas as pd
import os
import re
from huggingface_hub import InferenceClient
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import tiktoken
from tqdm import tqdm
import numpy
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv, find_dotenv
if not load_dotenv(find_dotenv()):
    raise Exception("Failed to load .env file")

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.getenv('OPENAI_API_KEY'),
                model_name="text-embedding-3-small"
            )
client = chromadb.PersistentClient("/home/yl3427/cylab/chroma_db")
client.list_collections()

[]

In [2]:
client.delete_collection("text-emb-3-small-n03-testing-split")

In [2]:
full_report_collection = client.get_or_create_collection("full_report_emb2")
full_report_collection.count()

In [3]:
summary_df = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/5_folds_summary/Updated_withSummary_merged_df.csv")
summary_df.columns

Index(['patient_filename', 't', 'text', 'type', 'n', 'prompt_for_abstract',
       'prompt_for_extract', 'abstractive_summary', 'extractive_summary',
       'zs_ans_from_text_t', 'zs_ans_from_abstract_t', 'zs_ans_from_extract_t',
       'zs_ans_from_text_n', 'zs_ans_from_abstract_n', 'zs_ans_from_extract_n',
       '0_Has_Valid_Prediction_t', '0_coded_pred_t',
       '1_Has_Valid_Prediction_t', '1_coded_pred_t',
       '2_Has_Valid_Prediction_t', '2_coded_pred_t',
       '0_Has_Valid_Prediction_n', '0_coded_pred_n',
       '1_Has_Valid_Prediction_n', '1_coded_pred_n',
       '2_Has_Valid_Prediction_n', '2_coded_pred_n', 'is_goodsum_t',
       'is_goodsum_n'],
      dtype='object')

In [4]:
def update(report_df, collection):
    pbar = tqdm(total=report_df.shape[0])

    for _, report in report_df.iterrows():
        report_text = report["text"]
        meta = {}
        meta["patient_filename"] = report["patient_filename"]
        meta["cancer_type"] = report["type"]
        meta["t_label"] = report['t']
        meta["n_label"] = report['n']
        meta['is_goodsum_t'] = report['is_goodsum_t']
        meta["is_goodsum_n"] = report["is_goodsum_n"]

        collection.update(
            ids=meta["patient_filename"],
            embeddings=openai_ef([report_text])[0],
            metadatas=meta,
            documents=report_text
        )
        pbar.update(1)
    pbar.close()

In [5]:
update(summary_df, full_report_collection)

  0%|          | 0/1517 [00:00<?, ?it/s]